In [167]:
import numpy as np
import pandas as pd
import pickle

In [168]:
# Function for filling ABV variable

def fill_ABV(df, varietal):
    
    """This function is used to fill the ABV values that are 0.0. It does it by 
    splitting off the Varietal information into two data frames, finding the mean and 
    standard deviation of the data that contains the ABV value, then filling the empty 
    values with a draw from a random distribution.
    
    Required Inputs:
    df: The pandas data frame containing the varietal and ABV information under columns
    'varietal' and 'ABV'
    
    varietal: The varietal that you are interested in filling with random draws.
    
    returns: A pandas data frame with the missing values for ABV for the varietal of
    interested filled with random draws."""
    
    VarietalDF = df.loc[df['varietal'] == varietal]
    AppendDF = df.loc[df['varietal'] != varietal]

    is_ABV = [a > 0 for a in VarietalDF.ABV]
    not_ABV = [a < 0.1 for a in VarietalDF.ABV]
    ValueDF = VarietalDF[is_ABV]
    ZeroDF = VarietalDF[not_ABV]

    mean = ValueDF.ABV.mean()
    std = ValueDF.ABV.std()
    ZeroDF['ABV'] = np.random.normal(loc=mean, scale=std)
    AppendDF = AppendDF.append(ValueDF)
    AppendDF = AppendDF.append(ZeroDF)
    return AppendDF

In [169]:
#Read in the Cab Sauv data and the rest of the data from the text file. 

WineDF = pd.read_csv('data/AllData.txt')


In [170]:
# Remove some of the extra information from when I poorly exported the CSV file

WineDF['price'] = WineDF['price'].str.replace('[', '')
WineDF['varietal'] = WineDF['varietal'].str.replace('\'', '')
WineDF['year'] = WineDF['year'].str.replace('\'', '')
WineDF['ABV'] = WineDF['ABV'].str.replace('\'', '')
WineDF['origin'] = WineDF['origin'].str.replace('\'', '') 
WineDF['origin'] = WineDF['origin'].str.replace(']', '') 
WineDF['origin2'] = WineDF['origin2'].str.replace('\'', '') 
WineDF['origin2'] = WineDF['origin2'].str.replace(']', '') 
WineDF['origin3'] = WineDF['origin3'].str.replace('\'', '') 
WineDF['origin3'] = WineDF['origin3'].str.replace(']', '') 
WineDF['origin4'] = WineDF['origin4'].str.replace('\'', '') 
WineDF['origin4'] = WineDF['origin4'].str.replace(']', '') 
WineDF['origin5'] = WineDF['origin5'].str.replace('\'', '') 
WineDF['origin5'] = WineDF['origin5'].str.replace(']', '') 

In [171]:
# Convert columns to the appropriate data types

WineDF['origin'] = pd.Series(WineDF['origin'], dtype='str')
WineDF['origin2'] = pd.Series(WineDF['origin2'], dtype='str')
WineDF['origin3'] = pd.Series(WineDF['origin3'], dtype='str')
WineDF['origin4'] = pd.Series(WineDF['origin4'], dtype='str')
WineDF['origin5'] = pd.Series(WineDF['origin5'], dtype='str')
WineDF['price'] = pd.Series(WineDF['price'], dtype='float')
WineDF['ABV'] = pd.Series(WineDF['ABV'], dtype='float')


In [172]:
# Get rid of leading whitespace in columns that have it

WineDF['year'] = WineDF['year'].str.lstrip()
WineDF['varietal'] = WineDF['varietal'].str.lstrip()
WineDF['origin'] = WineDF['origin'].str.lstrip()
WineDF['origin2'] = WineDF['origin2'].str.lstrip()
WineDF['origin3'] = WineDF['origin3'].str.lstrip()
WineDF['origin4'] = WineDF['origin4'].str.lstrip()
WineDF['origin5'] = WineDF['origin5'].str.lstrip()


In [173]:
# Find values in the year column that are numbers and drop all rows that contain strings. Then drop all years that 
# are not in the 1900s

is_year = [a.isdigit() for a in WineDF.year]
WineDF = WineDF[is_year]
WineDF['year'] = pd.Series(WineDF['year'], dtype='int')
WineDF = WineDF[WineDF['year'] > 1900]
# Drop high price

WineDF = WineDF[WineDF['price'] < 3000]
# Drop ABV value of that is too high  as there is one with ABV 83 and one with 20

WineDF = WineDF[WineDF['ABV'] < 20]

In [174]:
# Try to fill in ABV with a drawn from a normal distribution
varietals = WineDF.varietal.unique()
for varietal in varietals:
    WineDF = fill_ABV(WineDF, varietal)

/home/grant/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [175]:
WineDF = WineDF.reset_index()


In [176]:
# Try to get the outermost origin information. For most places it is country, except for the US where it is state.

columns = WineDF.columns[12:]

In [177]:
# Obtain the information about where the wine comes from. It is the outermost origin column for most wines,
# except using three extra California regions to help split the data some

California_regions = ['Central Coast', 'Sonoma County', 'Napa Valley']
origin = []
for wine in range (0, len(WineDF)):
    for ii in range(4,-1,-1):
        if ii == 0:
            origin.append(WineDF.at[wine, columns[ii]])
        elif WineDF.at[wine, columns[ii]] != 'nan':
            if WineDF.at[wine, columns[ii]] == 'California':
                area = 'California'
                for region in California_regions:
                    if WineDF.at[wine, columns[ii - 1]] == region:
                        area = (WineDF.at[wine, columns[ii-1]])
                origin.append(area)
                break
            origin.append(WineDF.at[wine, columns[ii]])
            break
                

In [178]:
# Create a new dataframe only containing one origin column

NewWineDF = WineDF[['price','varietal','year','ABV','is_green','is_collectable','is_boutique', 'is_screw', 'is_magnum', 'is_half', 'is_3L']]
NewWineDF['origin'] = origin


/home/grant/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [179]:
# Drop rows with 3 L bottles as there are not that many and drop 3L column

NewWineDF = NewWineDF[NewWineDF['is_3L'] == 0]
NewWineDF.drop(columns='is_3L', inplace = True)

In [180]:
# Remove quote character from the origin column

NewWineDF['origin'] = NewWineDF['origin'].str.replace('"', '')


In [181]:
# Turn the varietal and origin column into category in order to remove problems and merge categories.

NewWineDF['varietal'] = pd.Series(NewWineDF['varietal'], dtype='category')
NewWineDF['origin'] = pd.Series(NewWineDF['origin'], dtype='category')

In [182]:
# Drop these varietals as they are all small

NewWineDF = NewWineDF[NewWineDF['varietal'] != 'Rosé Sparkling Wine']
NewWineDF = NewWineDF[NewWineDF['varietal'] != 'Red Sparkling Wine']
NewWineDF = NewWineDF[NewWineDF['varietal'] != 'Non-Vintage Sparkling Wine']


In [183]:
# Turn it back into a string

NewWineDF['varietal'] = pd.Series(WineDF['varietal'], dtype='str')

In [184]:
# Create a series in order to find origin where the counts are < 1% of the data and merge into the other category

origin_series = NewWineDF.origin.value_counts()
origin_mask = (origin_series/origin_series.sum() * 100).lt(1)
# To replace df['column'] use np.where I.e 
NewWineDF['origin'] = np.where(NewWineDF['origin'].isin(series[mask].index),'Other',NewWineDF['origin'])

# FUTURE WORK: This would be a good place to play around with the origins. Maybe merging all the small 
# types is not the right approach, maybe merging the types from different continents (e.g. combine all
# South American wines into a single "South America" category) might be more useful!

In [185]:
# Get rid of White Zin as there are only a few occurences.
NewWineDF = NewWineDF[NewWineDF['varietal'] != 'White Zinfandel']

In [186]:
# Get dummy variables for Varietal and Origin

Varietals = pd.get_dummies(NewWineDF['varietal'])
Varietals = Varietals.drop('Pinot Noir',axis=1)

In [187]:
Origins = pd.get_dummies(NewWineDF['origin'])
Origins = Origins.drop('California',axis=1)

In [188]:
# Create new final data frame to use for analysis

FinalWineDF = NewWineDF[['price','year','ABV','is_green','is_collectable','is_boutique', 'is_screw', 'is_magnum', 'is_half']]
FinalWineDF = FinalWineDF.join(Varietals)
FinalWineDF = FinalWineDF.join(Origins)

In [189]:
FinalWineDF.shape


(8844, 36)

In [190]:
# Output dataframe to pickle to use for analysis

with open('data/CaliforniaFullDF.pickle', 'wb') as to_write:
    pickle.dump(FinalWineDF, to_write)